# Paid Search Bid Optimization

## Set-up:

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
import scipy.optimize as optimize
from scipy.optimize import curve_fit

## Part A:

First, let's load the data for the four keywords below.

In [2]:
keyword_1 = pd.read_csv('clicksdata.kw8322228.csv', index_col=0)
keyword_2 = pd.read_csv('clicksdata.kw8322392.csv', index_col=0)
keyword_3 = pd.read_csv('clicksdata.kw8322393.csv', index_col=0)
keyword_4 = pd.read_csv('clicksdata.kw8322445.csv', index_col=0)

Next, let's estimate the alpha and the beta for each of the four keywords.

First, we need to define the function used to run a nonlinear regression of clicks on bid value.

In [3]:
def compute_clicks(x, alpha, beta):
    return alpha*(1 - np.exp(-beta*x))

Next, we will use `scipy.optimize.curve_fit` to predict our unknown parameters for each of the four keywords.

In [4]:
# Initialize array for parameters of each of the four keywords
ab_params = np.zeros((4, 2))

# Isolate data for clicks and bid prices
x_values = pd.concat([keyword_1.iloc[:,0], keyword_2.iloc[:,0], keyword_3.iloc[:,0], keyword_4.iloc[:,0]], axis=1)
y_values = pd.concat([keyword_1.iloc[:,1], keyword_2.iloc[:,1], keyword_3.iloc[:,1], keyword_4.iloc[:,1]], axis=1)

# Iterate through each drug and perform nonlinear regression to get the parameter estimates
for i in range(4):
    opt, cov = curve_fit(f=compute_clicks, xdata=x_values.iloc[:,i], ydata=y_values.iloc[:,i], p0=(y_values.iloc[8][i], 1 / x_values.iloc[:,i].mean()))
    a_opt, b_opt = opt
    ab_params[i][0] = a_opt
    ab_params[i][1] = b_opt

df_ab_params = pd.DataFrame(ab_params, columns = ['alpha','beta'], index = ['kw8322228','kw8322392','kw8322393','kw8322445'])
df_ab_params

,alpha,beta
kw8322228,74.090862,0.039449
kw8322392,156.439802,0.150083
kw8322393,104.799293,0.079717
kw8322445,188.111279,0.432292


## Part B:

Now, let's load data on LTV dollar value and conversion rate values for each of the keywords.

In [5]:
xl_file = pd.ExcelFile('hw-kw-ltv-conv.rate-data.xlsx')
ltv_conv = xl_file.parse("Sheet1")
ltv_conv

,keyword,ltv,conv.rate
0,kw8322228,354,0.30
1,kw8322392,181,0.32
2,kw8322393,283,0.30
3,kw8322445,107,0.30


Using the alpha, beta parameters from Part A and the LTV and conversion rate values, let's estimate the optimal bids for each of the four keywords.

First, we need to construct a function that will compute profit as a function of bid prices, LTV, conversion rates, alpha, and beta. We will also create a function that will compute individual expenditure as a function of alpha, beta, and bid prices.

In [6]:
# Profit function
def negative_profit(x, alpha, beta, ltv, conv):
    return -(alpha*(1 - np.exp(-beta*x)))*(ltv*conv-x)

# Expenditure function
def ind_expenditure(x, alpha, beta):
    return x*alpha*(1 - np.exp(-beta*x))

Now let's use `scipy.optimize.minimize` to find the profit-maximizing bid price for each of the four keywords as well as the optimal profit and expenditure.

In [15]:
# Initialize array for profit-maximizing bid price for the four keywords
opt_bid_prices = np.zeros((4, 3))

# Set our bounds and initial values
lower_bound = 0
x0 = 40 
bounds_object = optimize.Bounds(lower_bound, np.inf)

# Iterate through each drug and find the optimal salespersons and profit
for i in range(4):
    tuple_values = (df_ab_params.iloc[i][0],df_ab_params.iloc[i][1],ltv_conv.iloc[i][1],ltv_conv.iloc[i][2])
    optimizer_output = optimize.minimize(negative_profit, x0, args=tuple_values, method='trust-constr', bounds=bounds_object, options={'verbose': 1})
    opt_bid_prices[i][0] = optimizer_output.x
    opt_bid_prices[i][1] = -optimizer_output.fun
    opt_bid_prices[i][2] = ind_expenditure(optimizer_output.x,df_ab_params.iloc[i][0],df_ab_params.iloc[i][1])

df_opt_bid = pd.DataFrame(opt_bid_prices, columns = ['Optimal bid price','Optimal profit','Optimal Expenditure'], index = ['kw8322228','kw8322392','kw8322393','kw8322445'])

C:\Users\andre\anaconda3\lib\site-packages\scipy\optimize\_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


`xtol` termination condition is satisfied.
Number of iterations: 72, function evaluations: 84, CG iterations: 60, optimality: 2.68e-06, constraint violation: 0.00e+00, execution time: 0.077 s.
`xtol` termination condition is satisfied.
Number of iterations: 82, function evaluations: 92, CG iterations: 70, optimality: 8.95e-06, constraint violation: 0.00e+00, execution time: 0.091 s.
`xtol` termination condition is satisfied.
Number of iterations: 73, function evaluations: 54, CG iterations: 61, optimality: 2.72e-06, constraint violation: 0.00e+00, execution time: 0.076 s.
`xtol` termination condition is satisfied.
Number of iterations: 85, function evaluations: 64, CG iterations: 73, optimality: 1.02e-05, constraint violation: 0.00e+00, execution time: 0.092 s.


Below, we can see our profit-maximizing bid price, optimal profit, and optimal expenditure obtained for each of the four keywords.

In [28]:
df_opt_bid

,Optimal bid price,Optimal profit,Optimal Expenditure
kw8322228,34.127622,3950.456957,1870.615458
kw8322392,13.563448,6032.902200,1844.754652
kw8322393,22.433868,5451.614108,1957.873584
kw8322445,5.816956,4544.188936,1005.718649


## Part C:

Assume that we now have a budget constraint of $3000 across the four keywords. Let's first set up our nonlinear constraint before we find the new optimal values

In [22]:
# Define total expenditure function
def total_expenditure(bids):
    total = []
    for i in range(len(bids)):
        expenditure = bids[i]*df_ab_params.iloc[i][0]*(1 - np.exp(-df_ab_params.iloc[i][1]*bids[i]))
        total.append(expenditure)
    return np.sum(total)

# Set nonlinear constraint
budget = 3000
budget_constraint_object = optimize.NonlinearConstraint(total_expenditure, 0, budget)

Next, let's extract the parameters of our keywords to their own arrays.

In [18]:
a_values = df_ab_params.iloc[:,0].to_numpy()
b_values = df_ab_params.iloc[:,1].to_numpy()
ltv_values = ltv_conv.iloc[:,1].to_numpy()
conv_values = ltv_conv.iloc[:,2].to_numpy()

Next, we need to create a new negative profit function that computes the negative of the profit by taking in the parameters of each of the four keywords as inputs.

In [21]:
def constrained_profit(x):
    negative_profits = []
    for i in range(len(x)):
        negative_prof = -(a_values[i]*(1 - np.exp(-b_values[i]*x[i])))*(ltv_values[i]*conv_values[i]-x[i])
        negative_profits.append(negative_prof)
    total_profit = np.sum(negative_profits)
    return total_profit

Now, let's re run our optimization with the new nonlinear constraint in place.

In [23]:
# Initialize array for profit-maximizing bid price for the four keywords
constr_bid_prices = np.zeros((4, 3))

# Set our bounds and initial values
lower_bound = 0
x0 = np.ones(4)*40
bounds_object2 = optimize.Bounds(lower_bound, np.inf)

# Iterate through each keyword and find the optimal values
optimizer_output2 = optimize.minimize(constrained_profit, x0, method='trust-constr', bounds=bounds_object2, constraints=budget_constraint_object)

C:\Users\andre\anaconda3\lib\site-packages\scipy\optimize\_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


From here, we can compute optimal bid amounts, profit, and expenditures for each keyword under this constraint.

In [27]:
# New optimal bid prices
constr_bid_prices[:,0] = optimizer_output2.x

# New profit values
for i in range(4):
    new_profit = -negative_profit(constr_bid_prices[i][0], df_ab_params.iloc[i][0],df_ab_params.iloc[i][1],ltv_conv.iloc[i][1],ltv_conv.iloc[i][2])
    constr_bid_prices[i][1] = new_profit
    
# New expenditure values
for i in range(4):
    new_expenditure = ind_expenditure(constr_bid_prices[i][0], df_ab_params.iloc[i][0],df_ab_params.iloc[i][1])
    constr_bid_prices[i][2] = new_expenditure
    
df_constr_bid = pd.DataFrame(constr_bid_prices, columns = ['Optimal bid price','Optimal profit','Optimal Expenditure'], index = ['kw8322228','kw8322392','kw8322393','kw8322445'])
df_constr_bid

,Optimal bid price,Optimal profit,Optimal Expenditure
kw8322228,17.924261,3315.507409,673.208970
kw8322392,8.118451,5487.232095,894.506851
kw8322393,12.828288,4836.614279,860.885343
kw8322445,3.775700,4286.482744,571.398834
